In [2]:
# Initial Setup.
import pandas as pd
import re
import numpy as np
import math
from google.colab import drive
import pandas as pd
import matplotlib.pyplot as plt
drive.mount('/content/gdrive')
path = '/content/gdrive/My Drive/Mastodon_SaiGinny/'

Mounted at /content/gdrive


In [6]:
data=pd.read_csv(path+'data_collection/0_355_normalized.csv',converters={'data.username': str,'data.id': str})

In [7]:
data.columns

Index(['data.verified', 'data.location', 'data.receives_your_dm',
       'data.protected', 'data.created_at', 'data.verified_type', 'data.name',
       'data.subscription_type', 'data.pinned_tweet_id',
       'data.profile_image_url', 'data.profile_banner_url', 'data.id',
       'data.username', 'data.most_recent_tweet_id', 'data.description',
       'data.public_metrics.followers_count',
       'data.public_metrics.following_count',
       'data.public_metrics.tweet_count', 'data.public_metrics.listed_count',
       'data.public_metrics.like_count', 'data.entities.description.mentions',
       'data.url', 'data.entities.url.urls',
       'data.entities.description.hashtags', 'data.entities.description.urls'],
      dtype='object')

In [8]:

data.head()

,data.verified,data.location,data.receives_your_dm,data.protected,data.created_at,data.verified_type,data.name,data.subscription_type,data.pinned_tweet_id,data.profile_image_url,...,data.public_metrics.followers_count,data.public_metrics.following_count,data.public_metrics.tweet_count,data.public_metrics.listed_count,data.public_metrics.like_count,data.entities.description.mentions,data.url,data.entities.url.urls,data.entities.description.hashtags,data.entities.description.urls
0,False,Berlin,False,False,2009-04-08T14:53:42.000Z,none,dmnk@infosec.exchange,NaN,1.388135e+18,https://pbs.twimg.com/profile_images/137940817...,...,4342,512,13399,69,25640,"[{'start': 51, 'end': 63, 'username': 'aflplus...",NaN,NaN,NaN,NaN
1,False,"Strasbourg, FR",False,False,2017-07-13T08:59:52.000Z,none,Claudia,NaN,1.831641e+18,https://pbs.twimg.com/profile_images/125161648...,...,2749,725,778,15,6693,"[{'start': 13, 'end': 24, 'username': 'CamBioc...",https://t.co/gsXVIt526i,"[{'start': 0, 'end': 23, 'url': 'https://t.co/...","[{'start': 26, 'end': 37, 'tag': 'ERCGrantee'}...",NaN
2,False,NaN,False,False,2014-02-09T17:45:54.000Z,none,Tero-Petri Ruoko,NaN,NaN,https://pbs.twimg.com/profile_images/719790981...,...,282,512,253,2,1405,NaN,NaN,NaN,NaN,NaN
3,True,"Twin Cities, Minnesota",False,False,2020-01-16T14:12:24.000Z,blue,Alan Rozenshtein,NaN,NaN,https://pbs.twimg.com/profile_images/121783634...,...,3935,628,5728,81,6816,"[{'start': 11, 'end': 26, 'username': 'UofMNLa...",https://t.co/GaCoPeTLKG,"[{'start': 0, 'end': 23, 'url': 'https://t.co/...",NaN,NaN
4,False,Gadfly in the Plutocracy,False,False,2021-03-23T18:58:46.000Z,none,Ashley M. Gjøvik,NaN,1.805006e+18,https://pbs.twimg.com/profile_images/164398319...,...,16315,3206,12004,95,29858,NaN,https://t.co/PURtXIXMpd,"[{'start': 0, 'end': 23, 'url': 'https://t.co/...",NaN,NaN


In [9]:
data.shape

(326, 25)

In [11]:
import requests
import os
import json
import pandas as pd
import time

# To set your enviornment variables in your terminal run the following line:
# export 'BEARER_TOKEN'='<your_bearer_token>'
bearer_token = 'AAAAAAAAAAAAAAAAAAAAADi9vgEAAAAALlGqsqiDThx1zhq1VNBmj1LY8c0%3DfxaDd86f5o1v6iBYeMCU9l0OwJPXm1KcIqEYqO2GgUYqAMTyto'


def create_url(id):
    # Specify the usernames that you want to lookup below
    # You can enter up to 100 comma-separated values.


    user_fields = "user.fields=description,created_at,public_metrics,location,url,verified,withheld"
    # User fields are adjustable, options include:
    # created_at, description, entities, id, location, name,
    # pinned_tweet_id, profile_image_url, protected,
    # public_metrics, url, username, verified, and withheld
    url = "https://api.twitter.com/2/users/{}/followers?user.fields=affiliation,connection_status,created_at,description,entities,id,location,most_recent_tweet_id,name,pinned_tweet_id,profile_banner_url,profile_image_url,protected,public_metrics,receives_your_dm,subscription_type,url,username,verified,verified_type,withheld&expansions=affiliation.user_id,most_recent_tweet_id,pinned_tweet_id&tweet.fields=article,attachments,author_id,card_uri,context_annotations,conversation_id,created_at,display_text_range,edit_controls,edit_history_tweet_ids,entities,geo,id,in_reply_to_user_id,lang,media_metadata,non_public_metrics,note_tweet,organic_metrics,possibly_sensitive,promoted_metrics,public_metrics,referenced_tweets,reply_settings,scopes,source,text,withheld".format(id)
    #url = "https://api.twitter.com/1.1/users/search.json?q={}&{}".format(usernames, user_fields)
    return url


def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """

    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2UserLookupPython"
    return r


def connect_to_endpoint(url):
    while True:
        response = requests.request("GET", url, auth=bearer_oauth)
        print(response.status_code)
        if response.status_code == 429:
            print("429 Too Many Requests - Waiting for 1 minute before retrying...")
            time.sleep(60)  # 1 minute
        elif response.status_code == 400:
            print("400 Bad Request")
            continue
        elif response.status_code != 200:
            raise Exception(
                "Request returned an error: {} {}".format(
                    response.status_code, response.text
                )
            )

        else:
            return response.json()


ids = data['data.id'][:2].tolist()
all_data = []

for id in ids:
    url = create_url(id)
    json_response = connect_to_endpoint(url)
    all_data.append(json_response)

df = pd.DataFrame(all_data)
df



403


Exception: Request returned an error: 403 {"client_id":"29277496","detail":"When authenticating requests to the Twitter API v2 endpoints, you must use keys and tokens from a Twitter developer App that is attached to a Project. You can create a project via the developer portal.","registration_url":"https://developer.twitter.com/en/docs/projects/overview","title":"Client Forbidden","required_enrollment":"Appropriate Level of API Access","reason":"client-not-enrolled","type":"https://api.twitter.com/2/problems/client-forbidden"}

In [ ]:
import requests
import time

response = requests.request("GET", url, auth=bearer_oauth)

rate_limit = response.headers.get('x-rate-limit-limit')
rate_remaining = response.headers.get('x-rate-limit-remaining')
rate_reset = response.headers.get('x-rate-limit-reset')
if rate_reset is not None:
    rate_reset_time = time.strftime('%Y-%m-%d %H:%M:%S', time.gmtime(int(rate_reset)))
else:
    rate_reset_time = 'N/A'

print(f"Rate Limit: {rate_limit}")
print(f"Remaining Requests: {rate_remaining}")
print(f"Rate Limit Resets At (UTC): {rate_reset_time}")

NameError: name 'url' is not defined

In [ ]:
df = pd.DataFrame(all_data)

In [ ]:
df.shape

(447, 2)

In [ ]:
df.to_csv(path+'3998_4445_raw.csv', index=False)

In [ ]:
len(df.loc[df['data'].isnull()])

37

In [ ]:
for i in df.loc[~df['data'].isnull()]['data']:
  print(i)

{'username': 'cathyhwang47', 'profile_image_url': 'https://pbs.twimg.com/profile_images/1614042780065480704/Ps4Fm97o_normal.jpg', 'subscription_type': 'None', 'verified': False, 'entities': {'url': {'urls': [{'start': 0, 'end': 23, 'url': 'https://t.co/5JTtpj0l5F', 'expanded_url': 'https://www.law.virginia.edu/faculty/profile/ch6cq/2914983', 'display_url': 'law.virginia.edu/faculty/profil…'}]}, 'description': {'urls': [{'start': 53, 'end': 76, 'url': 'https://t.co/xG9CGSB4wE', 'expanded_url': 'http://bit.ly/2kAqJD5', 'display_url': 'bit.ly/2kAqJD5'}], 'mentions': [{'start': 5, 'end': 12, 'username': 'UVALaw'}]}}, 'profile_banner_url': 'https://pbs.twimg.com/profile_banners/3195892692/1610738566', 'verified_type': 'none', 'description': 'Prof @UVALaw. I write about M&A deals and contracts: https://t.co/xG9CGSB4wE.', 'id': '3195892692', 'most_recent_tweet_id': '1841560132628693369', 'created_at': '2015-05-15T04:43:37.000Z', 'public_metrics': {'followers_count': 2768, 'following_count': 9

In [ ]:

df['index'] = df.index

if 'data' in df.columns and not df['data'].isnull().all():
    data_normalized = pd.json_normalize(df['data'].dropna()).set_index(df['index'][df['data'].notna()])
    data_normalized.columns = ['data.' + col for col in data_normalized.columns]
else:
    data_normalized = pd.DataFrame()

if 'includes' in df.columns and not df['includes'].isnull().all():
    includes_normalized = pd.json_normalize(df['includes'].dropna(), 'tweets').set_index(df['index'][df['includes'].notna()])

    if 'entities' in includes_normalized.columns:
        entities_normalized = pd.json_normalize(includes_normalized['entities'].dropna()).set_index(includes_normalized.index)
        entities_normalized.columns = ['includes.entities.' + col for col in entities_normalized.columns]
    else:
        entities_normalized = pd.DataFrame()

    if 'public_metrics' in includes_normalized.columns:
        public_metrics_normalized = pd.json_normalize(includes_normalized['public_metrics'].dropna()).set_index(includes_normalized.index)
        public_metrics_normalized.columns = ['includes.public_metrics.' + col for col in public_metrics_normalized.columns]
    else:
        public_metrics_normalized = pd.DataFrame()

    includes_normalized.columns = ['includes.' + col for col in includes_normalized.columns]
else:
    includes_normalized = pd.DataFrame()
    entities_normalized = pd.DataFrame()
    public_metrics_normalized = pd.DataFrame()

final_df = pd.concat([data_normalized, includes_normalized, entities_normalized, public_metrics_normalized], axis=1)

final_df.shape


(410, 25)

In [ ]:
final_df.columns

Index(['data.id', 'data.username', 'data.most_recent_tweet_id',
       'data.description', 'data.receives_your_dm', 'data.verified_type',
       'data.profile_image_url', 'data.profile_banner_url',
       'data.subscription_type', 'data.verified', 'data.protected',
       'data.created_at', 'data.name', 'data.entities.description.hashtags',
       'data.public_metrics.followers_count',
       'data.public_metrics.following_count',
       'data.public_metrics.tweet_count', 'data.public_metrics.listed_count',
       'data.public_metrics.like_count', 'data.url', 'data.pinned_tweet_id',
       'data.entities.url.urls', 'data.entities.description.urls',
       'data.location', 'data.entities.description.mentions'],
      dtype='object')

In [ ]:
len(final_df.loc[~final_df['data.pinned_tweet_id'].isnull()])

214

In [ ]:
final_df.to_csv(path+'3998_4445_normalized.csv', index=False)

In [ ]:
df['errors'][0]

[{'resource_id': '1649231552126091265',
  'parameter': 'pinned_tweet_id',
  'resource_type': 'tweet',
  'section': 'includes',
  'title': 'Authorization Error',
  'value': '1649231552126091265',
  'detail': 'Sorry, you are not authorized to see the Tweet with pinned_tweet_id: [1649231552126091265].',
  'type': 'https://api.twitter.com/2/problems/not-authorized-for-resource'}]

In [ ]:
all_data

Output hidden; open in https://colab.research.google.com to view.